In [121]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import itertools
from tqdm.contrib.concurrent import thread_map
import html5lib
import urllib.request, urllib.error
from concurrent.futures import ThreadPoolExecutor
import openpyxl


In [132]:

def get_all(i):
    url = "https://www.immoweb.be/en/search/house/for-sale?countries=BE&page="+ str(i) + "&orderBy=relevance"
    response = requests.get(url)
    soup = bs(response.text, 'html')

    urls = []
    for article in soup.find_all('iw-search-card-rendered'):
        for link in article.find_all('a'):
            urls.append(link.get('href'))
    return urls

def get_property(url):
    
    response = requests.get(url)
    broken_links = []
    if response.status_code != 200:
          broken_links.append(url)
          print(broken_links)
    else:
        df =  pd.concat(pd.read_html(response.text)).set_index(0).T 
        df['id'] = url.split('/')[-1]
        df = df.set_index('id')
        return df.loc[:, ~df.columns.duplicated()].copy()
print("ALL DONE")    

ALL DONE


In [133]:
NB_PAGES = 334

urls = list(itertools.chain.from_iterable(thread_map(get_all, range(1, NB_PAGES))))

100%|██████████| 333/333 [00:45<00:00,  7.27it/s]


In [ ]:
# with ThreadPoolExecutor() as pool:
#     results = list(pool.map(get_property, urls))
  

In [134]:
df1 = pd.DataFrame(pd.concat(thread_map(get_property, urls)))


 21%|██        | 2122/9990 [01:12<03:45, 34.83it/s]

['https://www.immoweb.be/en/classified/house/for-sale/lichtervelde/8810/10484888']


 54%|█████▎    | 5351/9990 [02:56<04:05, 18.86it/s]

['https://www.immoweb.be/en/classified/exceptional-property/for-sale/ezemaal/landen/3400/10552193']


 59%|█████▊    | 5865/9990 [03:15<02:14, 30.58it/s]

['https://www.immoweb.be/en/classified/mixed-use-building/for-sale/li%C3%A8ge/4000/10335997']


100%|██████████| 9990/9990 [05:30<00:00, 30.24it/s]


In [143]:
a = df1.columns
a

Index([          'Available as of',            'Available date',
       'Neighbourhood or locality',         'Construction year',
              'Building condition',     'Street frontage width',
             'Number of frontages',         'Surroundings type',
                   'Virtual visit',               'Living area',
       ...
               'Extra information',   'Current monthly revenue',
               'Age of annuitants',           'EPC description',
               'Percentage rented',         'Venue of the sale',
                  'Single session',            'Terms of visit',
                  'Starting price',                  'Sea view'],
      dtype='object', name=0, length=119)

In [144]:
b = df2.columns
b

Index([            'Available date',          'Construction year',
                            'Floor',           'Number of floors',
               'Building condition',        'Number of frontages',
           'Covered parking spaces',     'Outdoor parking spaces',
                'Surroundings type',                'Living area',
       ...
                'Extra information',                   'Sea view',
                'Number of annexes',            'EPC description',
                'Venue of the sale',             'Single session',
                   'Terms of visit',             'Starting price',
                'Age of annuitants', 'Professional space surface'],
      dtype='object', name=0, length=110)

In [150]:
for x in a:
    if x not in b:
        print(x)
        



Surface of the plot
Width of the lot on the street
Connection to sewer network
Gas, water & electricity
Land is facing street
Wooded land
Plot at rear
Flat land
Maximum duration of annuity
Current monthly revenue
Percentage rented


In [151]:
for y in b:
    if y not in a:
        print(y)

Floor
Number of floors


In [153]:
df1.info

<bound method DataFrame.info of 0                Available as of                Available date   
id                                                               
10572904  After signing the deed  September 30 2023 - 12:00 AM  \
10572903  After signing the deed  September 30 2023 - 12:00 AM   
10572509  After signing the deed      August 6 2023 - 12:00 AM   
10572508  After signing the deed  September 30 2023 - 12:00 AM   
10572922           To be defined                           NaN   
...                          ...                           ...   
10463456                     NaN                           NaN   
10570364  After signing the deed                           NaN   
10569762                     NaN                           NaN   
10571653                     NaN                           NaN   
10557917                     NaN                           NaN   

0        Neighbourhood or locality Construction year Building condition   
id                                

In [154]:
df2.info

<bound method DataFrame.info of 0                     Available date Construction year Floor Number of floors   
id                                                                              
10572051   October 1 2023 - 12:00 AM              2023     5                7  \
10572916  January 31 2024 - 12:00 AM              2023     3                4   
10572661      June 1 2023 - 12:00 AM              2020     1                6   
10572049   October 1 2023 - 12:00 AM              2023     6                6   
10572751                         NaN              1969    10               18   
...                              ...               ...   ...              ...   
10379741                         NaN               NaN   NaN                5   
10402797                         NaN              2023     2                1   
10391991                         NaN              2009   NaN              NaN   
10397730                         NaN              2022   NaN                3

In [155]:
df = pd.concat([df1, df2])

In [156]:
df.info

<bound method DataFrame.info of 0                Available as of                Available date   
id                                                               
10572904  After signing the deed  September 30 2023 - 12:00 AM  \
10572903  After signing the deed  September 30 2023 - 12:00 AM   
10572509  After signing the deed      August 6 2023 - 12:00 AM   
10572508  After signing the deed  September 30 2023 - 12:00 AM   
10572922           To be defined                           NaN   
...                          ...                           ...   
10379741                     NaN                           NaN   
10402797                     NaN                           NaN   
10391991                     NaN                           NaN   
10397730                     NaN                           NaN   
10289494                     NaN                           NaN   

0        Neighbourhood or locality Construction year Building condition   
id                                

In [157]:
df.to_excel('full.xlsx', index=True)